In [5]:
#Import the required library
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from io import StringIO
import time
import undetected_chromedriver as uc

In [6]:
seasons =["2013-2014", "2014-2015","2015-2016","2016-2017","2017-2018","2018-2019","2019-2020","2020-2021","2021-2022","2022-2023","2023-2024","2024-2025"]
base_url="https://www.capology.com/uk/premier-league/payrolls/{0}/"

In [11]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome(options=options)

driver.set_page_load_timeout(200)

with pd.ExcelWriter("Wages_Epl.xlsx") as writer:# visit your target site
    for season in seasons:
        target_url = base_url.format(season)
        #Updated scraping logic within your loop
        driver.get(target_url)    
        #
        wait = WebDriverWait(driver, 15)
        #
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#table tbody tr")))
        html_content = driver.page_source
    
        # Target the table by its ID 'table' as seen in your screenshot
        tables = pd.read_html(StringIO(html_content), attrs={'id': 'table'}, flavor="html5lib")
        df = tables[0]
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip() for col in df.columns.values]
            
        df = df.dropna(how='all', axis=1).dropna(how='all', axis=0)
        df.to_excel(writer, sheet_name=season, index=False)
        
# release the resources allocated by Selenium and shut down the browser
driver.quit()